In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import r2_score

In [2]:
meta = pd.read_csv("../input/meta_open.csv", index_col='uid', parse_dates=["datastart","dataend"], dayfirst=True)
temporal = pd.read_csv("../input/temp_open_utc_complete.csv", index_col='timestamp', parse_dates=True).tz_localize('utc')

In [3]:
buildingnames = temporal.columns[temporal.columns.str.contains("Office")]

In [4]:
# Import all models we are using
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import GradientBoostingRegressor
from  sklearn.linear_model import HuberRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import TheilSenRegressor
    
# Make array of models. Each model is an array of two elements.
# First element is a model-name, second is a model itself
models = [['AdaBoostRegressor', AdaBoostRegressor(n_estimators = 1000, random_state = 42)],
['BaggingRegressor', BaggingRegressor(n_estimators = 1000, random_state = 42)],
['DecisionTreeRegressor', DecisionTreeRegressor(random_state = 42)],
['DummyRegressor', DummyRegressor()],
['ExtraTreeRegressor', ExtraTreeRegressor(random_state = 42)],
['ExtraTreesRegressor', ExtraTreesRegressor(n_estimators = 1000, random_state = 42)],
['GaussianProcessRegressor', GaussianProcessRegressor(random_state = 42)],
['GradientBoostingRegressor', GradientBoostingRegressor(n_estimators = 1000, random_state = 42)],
['HuberRegressor', HuberRegressor()],
['KNeighborsRegressor', KNeighborsRegressor()],
['MLPRegressor', MLPRegressor(random_state = 42)],
['PassiveAggressiveRegressor', PassiveAggressiveRegressor(random_state = 42)],
['RANSACRegressor', RANSACRegressor(random_state = 42)],
['SGDRegressor', SGDRegressor(random_state = 42)],
['TheilSenRegressor', TheilSenRegressor(random_state = 42)]]

C:\Users\kitkat\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
MAPE_data = {}
RSQUARED_data = {}
NMBE_data = {}
CVRSME_data = {}
for elem in models:
    modelName = elem[0]
    model = elem[1]
    print('\n\n' + modelName + '\n_____________')
    for singlebuilding in buildingnames[:]:
        print("Modelling: " + singlebuilding)
        # Get Data
        single_timezone = meta.T[singlebuilding].timezone
        single_start = meta.T[singlebuilding].datastart
        single_end = meta.T[singlebuilding].dataend
        single_building_data = pd.DataFrame(temporal[singlebuilding].tz_convert(single_timezone).truncate(before=single_start,after=single_end))

        # Split into Training and Testing
        trainingdata = single_building_data[single_building_data.index.month.isin(["1","2","3","5","6","7","9","10","11"])]
        testdata = single_building_data[single_building_data.index.month.isin(["4","8","12"])]

        # Get weather file
        weatherfilename = meta.T[singlebuilding].newweatherfilename
        print("Weatherfile: "+weatherfilename)
        weather = pd.read_csv(os.path.join("../input/",weatherfilename),index_col='timestamp', parse_dates=True, na_values='-9999')
        weather = weather.tz_localize(single_timezone, ambiguous = 'infer')
        outdoor_temp = pd.DataFrame(weather[[col for col in weather.columns if 'Temperature' in col]]).resample("H").mean()
        outdoor_temp = outdoor_temp.reindex(pd.DatetimeIndex(start=outdoor_temp.index[0], periods=len(single_building_data), freq="H")).fillna(method='ffill').fillna(method='bfill')

        # Create training data array
        train_features = np.array(pd.concat([pd.get_dummies(trainingdata.index.hour),
                                             pd.get_dummies(trainingdata.index.dayofweek),
                   pd.Series(outdoor_temp[outdoor_temp.index.month.isin(["1","2","3","5","6","7","9","10","11"])].TemperatureC.values)], axis=1))
        train_labels = np.array(trainingdata[singlebuilding].values)

        # Create test data array
        test_features = np.array(pd.concat([pd.get_dummies(testdata.index.hour),
                                             pd.get_dummies(testdata.index.dayofweek),
                   pd.Series(outdoor_temp[outdoor_temp.index.month.isin(["4","8","12"])].TemperatureC.values)], axis=1))
        test_labels = np.array(testdata[singlebuilding].values)


        # Train the model on training data
        model.fit(train_features, train_labels);
        # Use the forest's predict method on the test data
        predictions = model.predict(test_features)

        # Calculate the absolute errors
        errors = abs(predictions - test_labels)
        # Calculate mean absolute percentage error (MAPE) and add to list
        MAPE = 100 * np.mean((errors / test_labels))
        NMBE = 100 * (sum(test_labels - predictions) / (pd.Series(test_labels).count() * np.mean(test_labels)))
        CVRSME = 100 * ((sum((test_labels - predictions)**2) / (pd.Series(test_labels).count()-1))**(0.5)) / np.mean(test_labels)
        RSQUARED = r2_score(test_labels, predictions)

        print("MAPE: "+str(MAPE))
        print("NMBE: "+str(NMBE))
        print("CVRSME: "+str(CVRSME))
        print("R SQUARED: "+str(RSQUARED))

        MAPE_data[singlebuilding] = MAPE
        NMBE_data[singlebuilding] = NMBE
        CVRSME_data[singlebuilding] = CVRSME
        RSQUARED_data[singlebuilding] = RSQUARED

        metrics = pd.DataFrame([MAPE_data, NMBE_data, CVRSME_data, RSQUARED_data]).T
        metrics.columns = ["MAPE", "NMBE", "CVRSME", "RSQUARED"]
        #metrics
        metrics.to_csv(modelName + '_metrics.csv')

    




AdaBoostRegressor
_____________
Modelling: Office_Cristina
Weatherfile: weather2.csv
MAPE: 27.086057495078965
NMBE: 1.0166219886345862
CVRSME: 28.589986502921278
R SQUARED: 0.18218026965653822
Modelling: Office_Jesus
Weatherfile: weather1.csv
MAPE: 143.98697775733626
NMBE: 4.097141967907661
CVRSME: 31.455292481301637
R SQUARED: 0.330548305538116
Modelling: Office_Jett
Weatherfile: weather1.csv
MAPE: 200.82928427401777
NMBE: -22.534278366798276
CVRSME: 67.12919901477767
R SQUARED: 0.12365754108326621
Modelling: Office_Jerry
Weatherfile: weather1.csv
MAPE: 102.80459905854877
NMBE: 1.6620851687294915
CVRSME: 49.85007877574114
R SQUARED: 0.2105826253285248
Modelling: Office_Lesa
Weatherfile: weather5.csv
MAPE: 46.40558187232992
NMBE: -1.0907865368435579
CVRSME: 42.32486162615919
R SQUARED: 0.11728965997397611
Modelling: Office_Jackie
Weatherfile: weather1.csv
MAPE: 1206.604994029668
NMBE: -91.24808835213003
CVRSME: 131.87445757288756
R SQUARED: -0.6485039564169401
Modelling: Office_Marla

MAPE: 27.880981365132467
NMBE: -3.3043807973754777
CVRSME: 29.12043316090792
R SQUARED: 0.6530592454089744
Modelling: Office_Mark
Weatherfile: weather3.csv
MAPE: 28.098587721722456
NMBE: -6.972823303922866
CVRSME: 28.596864339401783
R SQUARED: 0.30438095112691343
Modelling: Office_Travis
Weatherfile: weather8.csv
MAPE: 6.779867439599892
NMBE: -2.8026506940292912
CVRSME: 8.223528536321481
R SQUARED: 0.1255531059056697
Modelling: Office_Lena
Weatherfile: weather5.csv
MAPE: 34.56044387347841
NMBE: -8.829181770017438
CVRSME: 38.94772668916293
R SQUARED: 0.04023658480842918
Modelling: Office_Max
Weatherfile: weather3.csv
MAPE: 35.207898942018204
NMBE: -10.891615891674538
CVRSME: 34.0054767046065
R SQUARED: 0.20199620227553838
Modelling: Office_Gustavo
Weatherfile: weather5.csv
MAPE: 23.62548597147681
NMBE: -5.318414282865323
CVRSME: 27.36095859245353
R SQUARED: 0.16099860800176202
Modelling: Office_Penny
Weatherfile: weather4.csv
MAPE: 30.29491018666251
NMBE: -9.426072703895608
CVRSME: 30.7

MAPE: 38.02338582796501
NMBE: -7.767943970765985
CVRSME: 36.10202412368632
R SQUARED: 0.1106079965620792
Modelling: Office_Louise
Weatherfile: weather5.csv
MAPE: 28.273651298856866
NMBE: 0.48579144307968786
CVRSME: 29.664897569758764
R SQUARED: 0.20374060685795625
Modelling: Office_Guillermo
Weatherfile: weather5.csv
MAPE: 41.244313655602774
NMBE: -13.166499353648279
CVRSME: 35.51892657208418
R SQUARED: 0.14431562061004
Modelling: Office_Paulina
Weatherfile: weather4.csv
MAPE: 11.592925057022699
NMBE: -2.2909178921059583
CVRSME: 13.8563290657122
R SQUARED: 0.10797221989572114
Modelling: Office_Gabriela
Weatherfile: weather5.csv
MAPE: 19.312719750156074
NMBE: -6.773168720040054
CVRSME: 20.51244913502827
R SQUARED: 0.2767359720232577
Modelling: Office_Carolina
Weatherfile: weather2.csv
MAPE: 18.72710492345549
NMBE: -5.8270469337405935
CVRSME: 22.407689808413014
R SQUARED: -0.14939715255669106
Modelling: Office_Noel
Weatherfile: weather9.csv
MAPE: 11.074803326702604
NMBE: -3.2199053566368

MAPE: 17.858183102422007
NMBE: 0.4656609718135961
CVRSME: 21.38615456575262
R SQUARED: 0.5423904430701022
Modelling: Office_Jesus
Weatherfile: weather1.csv
MAPE: 131.01868771972406
NMBE: 8.172203866096709
CVRSME: 31.066091793328617
R SQUARED: 0.34701225309398087
Modelling: Office_Jett
Weatherfile: weather1.csv
MAPE: 158.81190780076116
NMBE: -7.481961867617028
CVRSME: 62.111402175497176
R SQUARED: 0.24977144409273566
Modelling: Office_Jerry
Weatherfile: weather1.csv
MAPE: 85.39565129410175
NMBE: 11.098350304959531
CVRSME: 45.251257227751736
R SQUARED: 0.34951650344504015
Modelling: Office_Lesa
Weatherfile: weather5.csv
MAPE: 32.0918399127119
NMBE: 10.308091658875862
CVRSME: 41.35153662022444
R SQUARED: 0.15742139899721663
Modelling: Office_Jackie
Weatherfile: weather1.csv
MAPE: 771.8123059513238
NMBE: -11.425222454859808
CVRSME: 63.32309515507236
R SQUARED: 0.6199043580419796
Modelling: Office_Marla
Weatherfile: weather3.csv
MAPE: 10.069630032974652
NMBE: 1.3691500992946386
CVRSME: 14.6

MAPE: 8.806490960346288
NMBE: -0.608863832447287
CVRSME: 13.737877654185494
R SQUARED: 0.839463450328696
Modelling: Office_Travis
Weatherfile: weather8.csv
MAPE: 3.8316390637177142
NMBE: -0.5136773582516031
CVRSME: 5.980937284050468
R SQUARED: 0.5374533618265149
Modelling: Office_Lena
Weatherfile: weather5.csv
MAPE: 30.66265999359449
NMBE: -4.132274618349356
CVRSME: 39.541374327025004
R SQUARED: 0.01075586638874615
Modelling: Office_Max
Weatherfile: weather3.csv
MAPE: 17.83656798383574
NMBE: -2.9587996487877337
CVRSME: 25.272428324917286
R SQUARED: 0.5592410188625555
Modelling: Office_Gustavo
Weatherfile: weather5.csv
MAPE: 12.363604269329697
NMBE: -6.689250140076347
CVRSME: 19.268909220966012
R SQUARED: 0.5838840862253294
Modelling: Office_Penny
Weatherfile: weather4.csv
MAPE: 13.599065940737095
NMBE: -5.262254761569205
CVRSME: 18.842152236805653
R SQUARED: 0.6808833429927617
Modelling: Office_Garrett
Weatherfile: weather5.csv
MAPE: 29.711831594312333
NMBE: -12.897041972358775
CVRSME:

...

Modelling: Office_Mark
Weatherfile: weather3.csv
MAPE: 17.76397934514432
NMBE: -1.42486688016461
CVRSME: 18.910453405088102
R SQUARED: 0.6958144919887583
Modelling: Office_Travis
Weatherfile: weather8.csv
MAPE: 4.7954325099280615
NMBE: 1.0761935914210667
CVRSME: 6.344921706846565
R SQUARED: 0.4794414704886172
Modelling: Office_Lena
Weatherfile: weather5.csv
MAPE: 22.848567157036857
NMBE: -1.980087651477714
CVRSME: 29.20403909587691
R SQUARED: 0.4603827727058263
Modelling: Office_Max
Weatherfile: weather3.csv
MAPE: 25.860786462816606
NMBE: -5.972654444938085
CVRSME: 25.963363947595226
R SQUARED: 0.5348113115450537
Modelling: Office_Gustavo
Weatherfile: weather5.csv
MAPE: 17.54769980997262
NMBE: -2.4138208770286274
CVRSME: 19.63957795569044
R SQUARED: 0.5677207739218361
Modelling: Office_Penny
Weatherfile: weather4.csv
MAPE: 24.09739120914372
NMBE: -11.876081324403811
CVRSME: 24.465582073277314
R SQUARED: 0.46197858485636356
Modelling: Office_Garrett
Weatherfile: weather5.csv
MAPE: 32.57

MAPE: 22.70152856188324
NMBE: 2.4708957415054704
CVRSME: 23.808543964507333
R SQUARED: 0.4870978313224178
Modelling: Office_Guillermo
Weatherfile: weather5.csv
MAPE: 21.601901013531627
NMBE: -10.166559762738855
CVRSME: 21.194755874922752
R SQUARED: 0.6953152086313446
Modelling: Office_Paulina
Weatherfile: weather4.csv
MAPE: 8.91227250172178
NMBE: -0.31301005482990346
CVRSME: 11.235982179106578
R SQUARED: 0.4134512874058599
Modelling: Office_Gabriela
Weatherfile: weather5.csv
MAPE: 13.313251176075717
NMBE: -4.384692084615457
CVRSME: 14.982331136409778
R SQUARED: 0.6141481254470404
Modelling: Office_Carolina
Weatherfile: weather2.csv
MAPE: 16.679308077750488
NMBE: -5.566779606369632
CVRSME: 20.06301200380417
R SQUARED: 0.07855755168042
Modelling: Office_Noel
Weatherfile: weather9.csv
MAPE: 61.13574904443128
NMBE: -34.647710142356416
CVRSME: 85.65861484717018
R SQUARED: -34.50757579284764
Modelling: Office_Alannah
Weatherfile: weather0.csv
MAPE: 33792663.838951886
NMBE: -10859616.70682753

MAPE: 143.94642722656306
NMBE: 9.890877806642619
CVRSME: 31.734906426762116
R SQUARED: 0.31859355990972216
Modelling: Office_Jett
Weatherfile: weather1.csv
MAPE: 155.10697001150243
NMBE: -1.8478891476247823
CVRSME: 58.236602080478875
R SQUARED: 0.34045720844386695
Modelling: Office_Jerry
Weatherfile: weather1.csv
MAPE: 110.87055170314632
NMBE: 16.905268152243654
CVRSME: 48.98924031438904
R SQUARED: 0.23761140202697817
Modelling: Office_Lesa
Weatherfile: weather5.csv
MAPE: 33.56284188385692
NMBE: 10.133716566388216
CVRSME: 37.13119429945592
R SQUARED: 0.32063220349660626
Modelling: Office_Jackie
Weatherfile: weather1.csv
MAPE: 694.2802893107136
NMBE: -8.029028108735748
CVRSME: 74.67736371747958
R SQUARED: 0.4713763645572271
Modelling: Office_Marla
Weatherfile: weather3.csv
MAPE: 8.837023643999633
NMBE: 2.853365529291921
CVRSME: 13.407946562041083
R SQUARED: 0.14712915295945916
Modelling: Office_Maryann
Weatherfile: weather3.csv
MAPE: 38.20358051917418
NMBE: -5.913022681802437
CVRSME: 34

MAPE: 4.70307303535834
NMBE: 0.5722011028441407
CVRSME: 6.214226469889161
R SQUARED: 0.5006659464820489
Modelling: Office_Lena
Weatherfile: weather5.csv
MAPE: 20.607361141183883
NMBE: 3.500618232952669
CVRSME: 28.55941974000915
R SQUARED: 0.48394175339550305
Modelling: Office_Max
Weatherfile: weather3.csv
MAPE: 24.89711624269968
NMBE: -3.7209968297459954
CVRSME: 25.462468064123414
R SQUARED: 0.552587392227531
Modelling: Office_Gustavo
Weatherfile: weather5.csv
MAPE: 17.915576912897972
NMBE: -3.7074679517207323
CVRSME: 19.69938025847266
R SQUARED: 0.5650841946962484
Modelling: Office_Penny
Weatherfile: weather4.csv
MAPE: 21.372006152483415
NMBE: -4.973224433041315
CVRSME: 22.270370175294733
R SQUARED: 0.554196648908069
Modelling: Office_Garrett
Weatherfile: weather5.csv
MAPE: 30.581033309371552
NMBE: -8.902402858359224
CVRSME: 34.749548370508876
R SQUARED: 0.30205187342893325
Modelling: Office_Annika
Weatherfile: weather0.csv
MAPE: 29.131320126784388
NMBE: 9.572563053432317
CVRSME: 36.7

MAPE: 8.75124937579149
NMBE: 0.3090620846887058
CVRSME: 11.071156475546202
R SQUARED: 0.43053376294104495
Modelling: Office_Gabriela
Weatherfile: weather5.csv
MAPE: 12.226527569726693
NMBE: 1.4150008441283617
CVRSME: 14.374620141679047
R SQUARED: 0.6448150254584196
Modelling: Office_Carolina
Weatherfile: weather2.csv
MAPE: 16.236533744904396
NMBE: -4.478256929266817
CVRSME: 19.707177742352304
R SQUARED: 0.11095280452259126
Modelling: Office_Noel
Weatherfile: weather9.csv
MAPE: 7.927512605464253
NMBE: -0.4550086896340992
CVRSME: 9.811074475787702
R SQUARED: 0.5341864977212527
Modelling: Office_Alannah
Weatherfile: weather0.csv
MAPE: 37.8171363396631
NMBE: 7.41836403891099
CVRSME: 41.9841160673515
R SQUARED: 0.29737493890622624
Modelling: Office_Aliyah
Weatherfile: weather0.csv
MAPE: 5.0825410134090285
NMBE: -0.36821119340077474
CVRSME: 6.576219778118195
R SQUARED: 0.6450249113453287
Modelling: Office_Curt
Weatherfile: weather2.csv
MAPE: 10.98877366552963
NMBE: -0.9646954936841888
CVRSME